In [120]:
def Strategy_1():
    import tushare as ts
    import os
    import datetime
    import pymongo
    import pandas

    client = pymongo.MongoClient("127.0.0.1", 27017)
    
    startDate = datetime.datetime.now() + datetime.timedelta(days=-60)
    stocks = client.Common.Stock.find({"foundTime": {"$lt": startDate}})
    threshold = 9.9
    today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    found = {}
    minDays = 1000

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({}, limit=4, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < 4:
            continue
        changes = []
        for i in range(3):
            if res[i+1]["close"] == 0:
                d = 0
            else:
                d = (res[i]["close"] / res[i+1]["close"] - 1) * 100
            changes.append(d)
        if changes[0] < threshold or (changes[1] <= threshold and changes[2] <= threshold):
            continue
        days = (today - res[3]["date"]).days
        if days > minDays:
            continue
        if days < minDays:
            minDays = days
            found = {}
        found[item["code"]] = res

    for k, v in found.iteritems():
        print k
        print pandas.DataFrame(v, columns=["date", "close", "volume"])
        print "\n"

    print("Done")

In [121]:
Strategy_1()

002181
        date  close    volume
0 2018-04-18   5.83  719360.0
1 2018-04-17   5.30  705618.0
2 2018-04-16   4.82  182746.0
3 2018-04-13   4.38   27875.0


002368
        date  close    volume
0 2018-04-18  35.07  315160.0
1 2018-04-17  31.88  264271.0
2 2018-04-16  33.07  309290.0
3 2018-04-13  30.06  186420.0


002806
        date  close   volume
0 2018-04-18  21.18  30429.0
1 2018-04-17  19.25  42294.0
2 2018-04-16  17.50  14771.0
3 2018-04-13  18.21  11297.0


300570
        date  close    volume
0 2018-04-18  22.94  220296.0
1 2018-04-17  20.85  203951.0
2 2018-04-16  18.95   36587.0
3 2018-04-13  18.57   39308.0


Done
